In [1]:
#import mariadb
import tqdm
import pickle
import pandas as pd
import mysql.connector
from decimal import Decimal
from neo4j import GraphDatabase
from datetime import timedelta

In [2]:
df_pw = pd.read_csv("pw.csv", delimiter=",")

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql
db_url = {
    'database': 'DarknetScrape',
    'drivername': 'mysql',
    'username': df_pw["name"][0],
    'password': df_pw["password"][0],
    'host': '127.0.0.1',
    'port': 3306,
    'query': {'charset': 'utf8'},
}

db_connection = create_engine(URL(**db_url), encoding="utf8")

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  


In [4]:
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [5]:
j_addresses=pd.read_csv('candidates_final.csv')
j_addresses=j_addresses.drop(columns=['Unnamed: 0'])
j_addresses

,Vendor,Address
0,CNSHarmony,34Xo7fKRe9MjYBf3tyJjAFg9qX2kuFuhA1
1,TheHoneymonster,1Ew7BnRn3Q3xkpKMpgc7r5YFrkQXpwXMz7
2,EUCarder,1Nou6Q9qmxbi48m6eSeYy7KHvSSiWFMq5y
3,kaliforniaProducts,12oCEud6XZm9ePxiUnbiUvsjUwEUs8GsgE
4,DerFeinschmecker,bc1q63eumgx8a0esl4uu2zdusheadf79nxg5zup8h0
...,...,...
194,GrandWizardsLair,1LoXrv7WQ5H6m2BdY99VpYsSdKcGyxwke4
195,clockwerk,14VSzUV8NffH3miGAGzdAx62gVumgw3afr
196,hashking,19bBP43Mh4edueHie5akFBiqRngjUcexeK
197,ChipChipperson,1LaxMbZ2zh6H7PuKFy5v9axCk1EPBpe8r4


In [6]:
df_sqlcsv = pd.read_csv('reviews_price_btc.csv')
df_sqlcsv.time = pd.to_datetime(df_sqlcsv.time)
df_sqlcsv["price_btc_reviews_satoshi"] = df_sqlcsv["price_btc_review"] * 100000000
df_sqlcsv = df_sqlcsv.sort_values(by="time",ascending=True)

In [7]:
query = """select a.vendor_id, b.username from (
SELECT vendor_id, count(*) as count
from vendors
group by vendor_id
) as a
JOIN (SELECT vendor_id, username from vendors where market = 'Cryptonia') b
on a.vendor_id = b.vendor_id
where a.count = 2;
"""
df = pd.read_sql(query, con=db_connection)
df=df[df['username'].isin(j_addresses['Vendor'])]
df_sqlcsv = df_sqlcsv[df_sqlcsv["vendor_id"].isin(df["vendor_id"])]
df_sqlcsv

,Unnamed: 0,price_eur,time,customer,vendor_id,vendor_review_id,price_btc_review,price_btc_reviews_satoshi
26608,26608,165.14,2017-10-09 14:35:07,tk [5],a081033ab42c85ae4101549cd64ff4cb,105344,0.040876,4.087624e+06
26607,26607,401.17,2017-10-19 19:25:48,tk [5],a081033ab42c85ae4101549cd64ff4cb,105343,0.083459,8.345885e+06
25551,25551,277.69,2017-10-20 13:31:27,I***5 [2],8438ba1a66f83580db67292ecced8f9c,104261,0.056041,5.604125e+06
25550,25550,104.36,2017-10-23 22:06:34,np [3],8438ba1a66f83580db67292ecced8f9c,104260,0.020842,2.084157e+06
26606,26606,110.91,2017-12-05 23:06:20,a**c [1],a081033ab42c85ae4101549cd64ff4cb,105342,0.011203,1.120269e+06
...,...,...,...,...,...,...,...,...
21974,21974,38.88,2019-07-29 08:43:06,c*4| [19],135137a5e88e1fa749c1374f51048773,100611,0.004563,4.562577e+05
22234,22234,20.73,2019-07-29 09:30:40,c** [17],1db74355bd33521d281b3e15cc6dbe21,100874,0.002433,2.432670e+05
24868,24868,18.23,2019-07-29 09:59:25,m**F [9],6233b71b602efb80ef560af4480af1fa,103560,0.002139,2.139295e+05
24020,24020,86.34,2019-07-29 10:01:51,w***g [15],5c6b492caa2e2af07032efbaa9a9dd46,102688,0.010132,1.013202e+06


In [8]:
escrow_addresses_v1 = pd.DataFrame(columns=['inDegree', 'outDegree', 'address','vendor'])
for i in df_sqlcsv.itertuples():
    neo_results = []
    neo_results2 = []
    start = str(i.time.date()- timedelta(days=3))
    end = str(i.time.date())
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    maxSum = str(round(i.price_btc_reviews_satoshi*1.4,0))
    neo_query = session.run("""
    WITH '""" + start + """' as startDate,  '""" + end + """' as endDate, """
    + minSum + """ as minSum, """ + maxSum +""" as maxSum
    MATCH (t:Transaction) 
        WHERE  t.inDegree = 1 and ( t.outDegree = 2 OR t.outDegree = 1 ) AND date(startDate) <= t.date <= date(endDate) 
        and minSum <= t.inSum<=maxSum
    WITH t, minSum, startDate, endDate
    MATCH (t)-[r:RECEIVES]->(a:Address),
        (t)-[r1:RECEIVES]->(c:Address)-[s:SENDS]->(t)
        WHERE minSum  <= r.value <= minSum * 1.05 AND c <> a AND  2000000<a.totalMoneyReceived<200000000 
        AND a.minDate>date('2017-09-08') AND a.maxDate<date('2019-07-31')
    WITH a, r, startDate, endDate
    MATCH (a)-[s:SENDS]->(mix:Transaction)
        WHERE date(startDate) <= mix.date <= date(endDate) AND r.value >= s.value 
    RETURN a, r.value
    """)
    for r in neo_query:
        neo_results2.append(r[0])
        neo_results.append(r[1])

    
    df_neo4j2 = pd.DataFrame(neo_results2)
    df_neo4j2["vendor"] = i.vendor_id
    df_neo4j2["review"] = i.vendor_review_id
    df_neo4j2["rvalue"] = pd.Series(neo_results)
    escrow_addresses_v1 = pd.concat([escrow_addresses_v1,df_neo4j2])
    
display(escrow_addresses_v1.head() )

In [9]:
# df_neo4jbig.to_csv("QueryResultsTransaction1Mix.csv")
# escrow_addresses_v1.to_csv("QueryResultsTransaction1EscrowActiveOnBoth0506.csv")

In [10]:
escrow_addresses_v1=pd.read_csv("QueryResultsTransaction1EscrowActiveOnBoth0506.csv")

In [11]:
escrow_addresses_v1

,Unnamed: 0,inDegree,outDegree,address,vendor,maxDate,minDate,totalMoneyReceived,review,rvalue
0,0,2,2,18Hw3c4WcfRW6cdtfSP8MDsaNHJbN8bpKa,a081033ab42c85ae4101549cd64ff4cb,2017-11-17,2017-10-06,4410000.0,105344.0,4180000.0
1,1,2,2,1BEVjYniwpY4itFtYLhautNpSJqDtPuYre,a081033ab42c85ae4101549cd64ff4cb,2017-10-06,2017-09-27,4571961.0,105344.0,4096761.0
2,2,1,1,15u1M5iN4HAivHSjbgd37RjapYjZwbALG7,a081033ab42c85ae4101549cd64ff4cb,2017-10-06,2017-10-06,4254815.0,105344.0,4254815.0
3,3,22,22,1MR4x3Q4AGUs2PSaXD3HTwV6wpJSky5sgr,a081033ab42c85ae4101549cd64ff4cb,2017-12-27,2017-10-01,64373825.0,105344.0,4218000.0
4,4,22,22,1MR4x3Q4AGUs2PSaXD3HTwV6wpJSky5sgr,a081033ab42c85ae4101549cd64ff4cb,2017-12-27,2017-10-01,64373825.0,105344.0,4218000.0
...,...,...,...,...,...,...,...,...,...,...
71073,6,24,24,3Mepvc6UtQeVqC2XVMbEhcAQUFKeAuC8Lg,5c6b492caa2e2af07032efbaa9a9dd46,2019-07-28,2019-03-01,51634708.0,102688.0,1058973.0
71074,7,4,4,1B4TGoHYBaNFeLGWriyVikWe6GqPe8fsAP,5c6b492caa2e2af07032efbaa9a9dd46,2019-07-29,2019-01-21,52408800.0,102688.0,1047700.0
71075,0,13,13,1AiGyCKxEReTGZfxzkVsAztpeiurnW9vm6,927b6d058152a78d2b3fd80bd12d5778,2019-07-27,2019-07-27,17375144.0,104324.0,444000.0
71076,1,13,13,1AiGyCKxEReTGZfxzkVsAztpeiurnW9vm6,927b6d058152a78d2b3fd80bd12d5778,2019-07-27,2019-07-27,17375144.0,104324.0,444000.0


In [12]:
len(escrow_addresses_v1['address'].unique())

16553

In [13]:
v2=escrow_addresses_v1.groupby(by='address').count().sort_values(by=['inDegree'], ascending=False)
# v2=v2[v2['inDegree']>20]
v2=v2.reset_index()
escrow_addresses_v2=v2['address'].tolist()

In [14]:
escrow_addresses_v2_withUsername=escrow_addresses_v1[escrow_addresses_v1['address'].isin(escrow_addresses_v2)]
escrow_addresses_v2_withUsername=escrow_addresses_v2_withUsername[['address', 'vendor']]
escrow_addresses_v2_withUsername=escrow_addresses_v2_withUsername.merge(df, how="inner", left_on='vendor', right_on='vendor_id')
escrow_addresses_v2_withUsername=escrow_addresses_v2_withUsername.drop_duplicates()
escrow_addresses_v2_withUsername

,address,vendor,vendor_id,username
0,18Hw3c4WcfRW6cdtfSP8MDsaNHJbN8bpKa,a081033ab42c85ae4101549cd64ff4cb,a081033ab42c85ae4101549cd64ff4cb,SteroidWarehouse
1,1BEVjYniwpY4itFtYLhautNpSJqDtPuYre,a081033ab42c85ae4101549cd64ff4cb,a081033ab42c85ae4101549cd64ff4cb,SteroidWarehouse
2,15u1M5iN4HAivHSjbgd37RjapYjZwbALG7,a081033ab42c85ae4101549cd64ff4cb,a081033ab42c85ae4101549cd64ff4cb,SteroidWarehouse
3,1MR4x3Q4AGUs2PSaXD3HTwV6wpJSky5sgr,a081033ab42c85ae4101549cd64ff4cb,a081033ab42c85ae4101549cd64ff4cb,SteroidWarehouse
5,17MfrdMCjRLi9SVzNKfEAWN4xqPQB8xyTi,a081033ab42c85ae4101549cd64ff4cb,a081033ab42c85ae4101549cd64ff4cb,SteroidWarehouse
...,...,...,...,...
71073,13yGuUscfPEzEarfLUSmPVjXkskVLV1QBV,522610744070f63e75620b51cf3b7612,522610744070f63e75620b51cf3b7612,somecvvvendor
71074,3H2sfCtfnCJ4GusmyhpqzqV4XrnHgvynVL,522610744070f63e75620b51cf3b7612,522610744070f63e75620b51cf3b7612,somecvvvendor
71075,1G15iTrk8pjh11pbdmyfmNKNFbdPGcRmYW,522610744070f63e75620b51cf3b7612,522610744070f63e75620b51cf3b7612,somecvvvendor
71076,337Qkzp85skXra8M1fTJMiJRXBxxb2V9y5,522610744070f63e75620b51cf3b7612,522610744070f63e75620b51cf3b7612,somecvvvendor


In [15]:
escrow_addresses_v3= pd.DataFrame(columns=['escrow', 'vendor', 'relationships'])
for row in escrow_addresses_v2_withUsername.itertuples():
    vendor=j_addresses[j_addresses['Vendor']==row.username]['Address'].iloc[0]
  
    neo_query = session.run("""
    MATCH p=(escrow:Address {address: '"""+row.address+"""' })-[:RECEIVES|SENDS*2..10]->(vendor:Address {address: '"""+ vendor+"""'})
    RETURN collect(escrow) as escrow, collect(vendor) as vendor, collect(last(relationships(p))) as relationships""")
    for r in neo_query:
        escrow=r[0]
        vendor=r[1]
        relationships=r[2]
        escrow_addresses_v3.append({'escrow': escrow, 'vendor':vendor, 'relationships':relationships}, ignore_index=True)

display(escrow_addresses_v3.head())

No data
Failed to read from defunct connection IPv4Address(('127.0.0.1', 7687)) (IPv4Address(('127.0.0.1', 7687)))


SessionExpired: Failed to read from defunct connection IPv4Address(('127.0.0.1', 7687)) (IPv4Address(('127.0.0.1', 7687)))

In [16]:
escrow_addresses_v3

,escrow,vendor,relationships


In [17]:
row

Pandas(Index=0, address='18Hw3c4WcfRW6cdtfSP8MDsaNHJbN8bpKa', vendor='a081033ab42c85ae4101549cd64ff4cb', vendor_id='a081033ab42c85ae4101549cd64ff4cb', username='SteroidWarehouse')

In [ ]:
escrow_addresses_v3.to_csv('escrow_addresses_v3.csv')

In [ ]:
session.close()